<div style="color:white;
           display:fill;
           border-radius:25px;
           background-color:Blue;
           font-size:210%;
           font-family:Verdana;
           letter-spacing:0.5px">
<p style="padding: 10px;
          color:white;
          text-align:center;"
          >
       WELCOME TO MY NOTEBOOK
</p>
</div>

# About Dataset: Digit Recognizer
The dataset consists of two files, "train.csv" and "test.csv," containing grayscale images of **handwritten digits ranging from zero to nine**. Each image is a 28x28 pixel grid, totaling 784 pixels. Each pixel is represented by an integer value between 0 and 255, indicating its darkness (higher values mean darker). Our main goal is to classify the digit based on the input(that is in the form of digit image) using Maching Learning Model. We use the **Convolutional Neural Network** to classify the digits images.


![](https://media.tenor.com/IZ2eQ1jjxdAAAAAd/numbers.gif)



Thank you for exploring my notebook. Please take a moment to upvote my notebook. Your support motivates me to keep improving and sharing valuable insights.😊

# Import all the libraries

In [ ]:
# import all the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Read the Train and Test data

In [ ]:
train= pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
train.head()

# Check the Shape of Dataset

In [ ]:
# Check the shape of the train data
train.shape

In [ ]:
# Getting x_train and y_train
x_train= train.drop("label", axis=1)
y_train=train["label"]

In [ ]:
# check the shape of the data 
x_train.shape, y_train.shape

# Data Preprocessing

In [ ]:
# rescale the pixels between 0 and 1
x_train= x_train/255.0

# Reshaping the Train Data

In [ ]:
# reshape the training data
x_train= x_train.values.reshape((-1, 28, 28, 1))
x_train.shape

# Lets visualize the training data 

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(20):
    ax = plt.subplot(8, 4, i + 1)
    plt.imshow(x_train[i])
    plt.title(f"Title:{y_train[i]}")
    plt.axis("off")

# Data Augmentation
> Data Augmentation is a technique which is used to increase the diversity in the dataset by applying different types of transformations to the original data.The main goal of data augmentation is to create a new training examples that are the variations of the existing data without changing the underlying target or label value.This helps improve the performance and robustness of the machine learning model.

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                          samplewise_center=False,
                                                          rotation_range=10,
                                                          width_shift_range=0.2,
                                                          height_shift_range=0.2,
                                                          zoom_range=0.1
                                                         )
datagen.fit(x_train)                                

In [ ]:
# Apply data augmentation to the training data
train_gen= datagen.flow(x_train, y_train, batch_size=32)

# Lets Create the CNN Model

In [ ]:
# Create the model

tf.random.set_seed(42)

def get_model():
    model= tf.keras.Sequential([
                             tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu", input_shape=(28, 28,1), padding="same"), # first layer specifies input shape (height, width, colour channels)
                             tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
                             tf.keras.layers.MaxPool2D(pool_size=2),
                             tf.keras.layers.BatchNormalization(),

                             tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
                             tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
                             tf.keras.layers.MaxPool2D(2),
                             tf.keras.layers.BatchNormalization(),

                             tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
                             tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
                             tf.keras.layers.MaxPool2D(2),
                             tf.keras.layers.BatchNormalization(),

                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation="relu"),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(10, activation="softmax") 
        
        
    
    ])
    
    return model

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
# Compile the model
    model = get_model()
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), # different loss function for multiclass classifcation
                  optimizer=tf.keras.optimizers.Adam(lr=0.001),
                  metrics=["accuracy"]
                 )


# Callbacks--> Stop training automatically once the model performance stop improving
    learning_rate_reduction=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',patience=2,factor=0.5, min_lr = 0.00001,verbose =1)
    Early_Stopping= tf.keras.callbacks.EarlyStopping(monitor="loss", patience=500, restore_best_weights=True)

    # Fit the model
    model.fit(train_gen, epochs=100, callbacks=[learning_rate_reduction,Early_Stopping], verbose=1)

# Model Summary

In [ ]:
model.summary()

# Read the Test Data

In [ ]:
test_data.head()

In [ ]:
test_data.shape

# Preprocess the Test Data and Rescale it Between 0 and 1

In [ ]:
test_data=test_data/255.0
test_data= test_data.values.reshape((-1, 28, 28, 1))
y_preds=model.predict(test_data)
y_preds=y_preds.argmax(axis=1)

In [ ]:
# generate the index
index=[]
for i in range(1,28001):
    index.append(i)

In [ ]:
# store the resluts into dataframe
df= pd.DataFrame(index)
df["ImageId"]=index
df["Label"]=y_preds
df.head()

In [ ]:
df.drop([0], axis=1, inplace=True)

In [ ]:
# Save the predictions
df.to_csv("result_final.csv", index= False)